# Flame Speed

In this example we simulate a freely-propagating, adiabatic, 1-D flame and
* Calculate its laminar burning velocity
* Perform a sensitivity analysis of its kinetics

The figure below illustrates the setup, in a flame-fixed coordinate system. The reactants enter with density $\rho_{u}$, temperature $T_{u}$ and speed $S_{u}$. The products exit the flame at speed $S_{b}$, density $\rho_{b}$ and temperature $T_{b}$.

<img src="./images/flameSpeed.png" alt="Freely Propagating Flame" style="width: 300px;"/>

In [157]:
# Add python code here

## Cantera Simulation Steps

Most Cantera simulations are accomplished by three steps:

1. Create a phase from an input file
2. Set boundary/input conditions
3. Run the simulation

In the case of an adiabatic free flame, Cantera has a built-in model to quickly calculate flame speeds.

### Define the reactant conditions, gas mixture and kinetic mechanism associated with the gas

In [158]:
# Add python code here

### Define flame simulation conditions

In [159]:
# Add python code here

### Run the simulation

With the input conditions set, we need to create the appropriate flame object and run the simulation. The `FreeFlame` class can take either an array of grid points or a width. Specifying the width is preferred and Cantera will automatically set and refine a grid in the simulation.

In [160]:
# Add python code here

In [161]:
# Add python code here

With 1-D flames, we need to consider species and energy transport by convection and diffusion. For species diffusion, there are several ways of calculating the binary diffusion coefficient of every pair of species. The simpler assumption is that the species is diffusing into an average mixture. The more complicated, but more accurate, assumption is to calculate the multi-component diffusion coefficients.

In this example, we are using the mixture-average assumption, with the `"Mix"` keyword. Using the `"Multi"` solution can substantially increase the time it takes to reach a solution.

However, you can start by solving the system with the mixture-average assumption and switch to the multicomponent assumption after a good initial solution has already been achieved.

In [162]:
# Add python code here

The auto option in the solve function tries to "automatically" solve the flame by applying a few common techniques. First, the flame is solved on a sparse grid with the transport calculations set to mixture averaged. Then grid refinement is enabled, with points added according to the values of the ratio, slope, and curve parameters in the set_refine_criteria function. If the initial solve on the sparse grid fails to converge, the simulation is attempted again, but this time with the energy equation disabled. Once the simulation has been solved on the refined grid with the mixture averaged transport, Cantera enables the multicomponent transport and Soret diffusion, if they have been set by the user.

In general, it is recommended that you use the auto option the first time you run the solver, unless the simulation fails. On subsequent invocations of solve, you should not include the auto option (or set it to False).

In [163]:
# Add python code here

Cantera can automatically refine the solution grid to meet certain criteria. 

In [164]:
# Add python code here

The four refinement parameters are `ratio`, `slope`, `curve`, and `prune`.

   * `ratio` limits the maximum distance between two grid points
   * `slope` adds grid points where the first derivative of the solution exceeds the threshold
   * `curve` adds grid points where the second derivative of the solution exceeds the threshold
   * `prune` will remove unnesseccary grid points in regions where the solution is over-refined (setting prune to 0.0 will preserve all solution points)


In [165]:
# Add python code here

Cantera's 1-D solver can produce several levels of output, depending on how much detail you want to see. If you're pretty sure a solution is going to work, then a `loglevel` of 0 (no output) or 1 (minimal output) will be appropriate. If a case is failing, you can increase `loglevel` up to a maximum of 8 to have more and more output from each solution step.

In [166]:
# Add python code here

Create a new flame object and set a higher refinement criteria to observe the difference in solution time and flamespeed.

In [167]:
# Add python code here

Set the new refinement criteria and solve.

In [168]:
# Add python code here

### Plot figures

Check and see if all has gone well. Plot temperature and species fractions to see. We expect that the solution at the boundaries of the domain will have zero gradient (in other words, that the domain width that we specified is wide enough for the flame). Notice that Cantera automatically expanded the domain from 2 cm to 4 cm to accommodate the flame thickness.

#### Temperature Plot

In [169]:
# Add python code here

#### Major species plot

In [170]:
# Add python code here

#### Comparing Refinement Levels

In [171]:
# Add python code here

## Sensitivity analysis
Compute normalized sensitivities of flame speed $S_u$ to changes in the rate coefficient $k_i$ for each reaction
$$s_i = \frac{k_i}{S_u} \frac{d S_u}{d k_i}$$

Note that this will be much slower when multicomponent or Soret diffusion are turned on.

In [172]:
# Add python code here

In [173]:
# Add python code here

In [174]:
# Add python code here

## Solving multiple flames (parameter sweep) 

Cantera also makes it easy to re-use solutions from previous flames to compute conditions for a similar flame. This is very useful when doing a parameter sweep. In this case, we are going to sweep over a range of equivalence ratios. We will start at the lower limit of the equivalence ratio range we are interested in, 0.6.

In [175]:
# Add python code here

In the grid refinement criteria, it is important that we add one more condition, `prune`. This parameter controls when grid points can be removed from the simulation. Since we are conducting a sweep of equivalence ratios here, we expect the flame thickness to vary so that the number of grid points necessary will vary as well. Without `prune`, the number of grid points could never decrease and it would slow down some of the solutions.

In [176]:
# Add python code here

Now we will solve the flame. For this first case, we are going to set `auto=True`.

In [177]:
# Add python code here

Now we will construct the range of equivalence ratios to loop over. Notice that the rest of these solutions are conducted with `auto=False`, since we are starting from a known-good solution.

In [178]:
# Add python code here

In [179]:
# Add python code here

## Challenge question: Evaluating NO<sub>x</sub> formation for freely propagating flames
Nitrogen oxides or NO<sub>x</sub> is a unavoidable pollutant for any combustion device that uses air as an oxidizer. NO<sub>x</sub> refers to two nitrogen compounds:
 * NO<sub>2</sub>: yellow/brown corrosive, toxic gas responsible for photochemical smog and ozone depletion
 * NO: A toxic gas which reacts with ozone to produce NO<sub>2</sub>
 
NO<sub>x</sub> is essentially unavoidable in any combustion system that uses air as an oxidizer, as the presence of oxygen and nitrogen at high temperatures will produce NO<sub>x</sub>. When designing a combustion device, care must be taken to minimize the amount of NO<sub>x</sub> formed, and NO<sub>x</sub> emissions are heavily regulated.

To better understand how to mitigate NO<sub>x</sub> formation, and to test your skills learned in this lesson, write a code that will record and plot the NO and NO<sub>2</sub> mole fractions at the outlet from your combustor (the final point in your flame solution). Continue to use methane as fuel and air as the oxidizer, as well as the GRI 3.0 chemical kinetics model. In particular, you should evaluate the effect of varying:
 * Equivalence ratio
 * Ambient pressure Po
 * Fresh fuel/oxidizer mixture temperature To
 
# Bonus
Extend your code to also consider the effect of residence time on mole fractions of NO and NO<sub>2</sub>

Hints:
 * Residence time can be calculated from flame.grid and flame.velocity
 * Make the solution domain larger to allow for longer residence times